In [8]:
import pandas as pd
import unicodedata #permet d'enlever les accents
import re #Pour utiliser des expressions régulières

# 1. Chargement des données et nettoyage

### Importation des données des cinq fichiers

In [12]:
type_df = pd.read_excel('data_tp/type.xlsx')
caracteristiques_df = pd.read_excel('data_tp/caracteristiques.xlsx')
etablissements_df = pd.read_excel('data_tp/etablissements.xlsx')
territoires_df = pd.read_excel('data_tp/territoires.xlsx')
adresses_df = pd.read_excel('data_tp/adresses.xlsx')

### Analyse de caracteristiques_df

In [15]:
# Vérification des valeurs manquantes
missing_values_type_df = type_df.isnull().sum()
print("Nombre de valeurs manquantes par colonne pour types_df:")
print(missing_values_type_df)
caracteristiques_df

Nombre de valeurs manquantes par colonne pour types_df:
ETBL_ID                 0
ETBL_TYPE_GENRE         0
ETBL_TYPE_ID         1370
ETBL_TYPE_FR         1370
ETBL_TYPE_EN         1370
ETBL_TYPE_GRP_ID     9380
ETBL_TYPE_GRP_FR     9380
ETBL_TYPE_GRP_EN     9380
ETBL_TYPE_CATG_ID       0
ETBL_TYPE_CATG_FR       0
ETBL_TYPE_CATG_EN       0
dtype: int64


,ETBL_ID,CARACT_ID,CARACT_NOM_FR,CARACT_NOM_EN,CARACT_ATTRB_ID,CARACT_ATTRB_NOM_FR,CARACT_ATTRB_NOM_EN,CARACT_ATTRB_VAL
0,838721,10581,Activités,Activities,20143552.0,Golf à moins de 10 km,Golf within10 km,NaN
1,838721,10581,Activités,Activities,20143561.0,Ski de fond à moins de 10 km,Cross-country skiing within 10 km,NaN
2,838721,10581,Activités,Activities,20143568.0,Théâtre d'été à moins de 10 km,Summer theatre within 10 km,NaN
3,838721,10581,Activités,Activities,10667.0,Vélo,Cycling,NaN
4,838721,10579,Services,Services,364299662.0,Accès Internet : sans fil gratuit,Internet access : free wireless,NaN
...,...,...,...,...,...,...,...,...
341316,500016738,14372,Équipement des unités,Amenities,433306.0,Unité pour non-fumeur,Non-smoking units,1.0
341317,500016738,16957126,Unités,Units,16957153.0,"appartement~, chalet~ ou maison~","apartment~, cottage~ or house~",1.0
341318,500016738,10497,Classification,Rating,18336.0,En cours d'évaluation,Rating in progress,NaN
341319,500016738,420859,Tarifs,Rates,18252.0,Maximum pour l'unité la moins chère,Maximum for the least expensive unit,435.0


### Analyse de etablissements_df 

In [18]:
missing_values_etablissements_df = etablissements_df.isnull().sum()
print("Nombre de valeurs manquantes par colonne pour etablissements_df:")
print(missing_values_etablissements_df)
# Récupération des types uniques dans la colonne ETBL_DESC_FR sans NaN
types_uniques = etablissements_df['ETBL_DESC_FR'].dropna().apply(type).unique()
print('Les types uniques dans la colonne ETBL_DESC_FR : ')
print(types_uniques)

Nombre de valeurs manquantes par colonne pour etablissements_df:
ETBL_ID               0
ETBL_NOM_FR           0
ETBL_NOM_EN           0
ETBL_DESC_FR       4536
ETBL_DESC_EN       4544
ETBL_RESERVABLE       0
dtype: int64
Les types uniques dans la colonne ETBL_DESC_FR : 
[<class 'str'>]


#### Les colonnes ETBL_DESC_FR et ETBL_DESC_EN contiennent des chaînes de caractères, donc je remplace les NaN par des chaînes vides.

In [26]:
etablissements_df.columns

etablissements_df['ETBL_DESC_FR'].fillna('', inplace=True)  # Remplacer par une chaîne vide
etablissements_df['ETBL_DESC_EN'].fillna('', inplace=True)

### Analyse de territoires_df

In [28]:
territoires_df

,ETBL_ID,TERR_GENRE,TERR_ZONE_ID,TERR_ZONE_NOM_FR,TERR_ZONE_NOM_EN,TERR_ZONE_GENRE_ID,TERR_ZONE_GENRE_FR,TERR_ZONE_GENRE_EN
0,838721,principal,292120028,Le mont Royal,The Mont Royal,432971,Quartier / secteur,District / Area
1,838721,principal,21950,Montréal,Montréal,91,Ville,City
2,838721,principal,3432,Montréal,Montréal,81,Région,Area
3,838721,principal,3406,Québec,Quebec,75,Province,Province
4,838774,principal,22830,Boileau,Boileau,91,Ville,City
...,...,...,...,...,...,...,...,...
37173,500016737,principal,3430,Laurentides,Laurentides,81,Région,Area
37174,500016737,principal,3406,Québec,Quebec,75,Province,Province
37175,500016738,principal,21384,Saint-Irénée,Saint-Irénée,91,Ville,City
37176,500016738,principal,3414,Charlevoix,Charlevoix,81,Région,Area


In [30]:
territoires_df.isnull().sum()

ETBL_ID               0
TERR_GENRE            0
TERR_ZONE_ID          0
TERR_ZONE_NOM_FR      0
TERR_ZONE_NOM_EN      0
TERR_ZONE_GENRE_ID    0
TERR_ZONE_GENRE_FR    0
TERR_ZONE_GENRE_EN    0
dtype: int64

Pas de valeurs NaN dans territoires_df 

# 2. Création trois nouvelles variables

In [22]:
#Fonction pour enlever les accents
def enlever_accents(chaine):
    if isinstance(chaine, str):  # Vérifier si la valeur est une chaîne
        return ''.join(c for c in unicodedata.normalize('NFD', chaine) if unicodedata.category(c) != 'Mn')
    return chaine  # Retourne la valeur inchangée si ce n'est pas une chaîne
    
# Appliquation de la fonction à la colonne ETBL_DESC_FR
etablissements_df['ETBL_DESC_FR'] = etablissements_df['ETBL_DESC_FR'].apply(enlever_accents)

### Ajout des nouvelles colonnes avec les conditions demandées

\b : délimite le mot, assurant que la correspondance se fait sur des mots entiers.

(s)? : capture le "s" optionnel pour inclure les formes pluriel des mots.

In [30]:
# Création de nouvelles colonnes avec des valeurs 0/1 selon la présence des mots
etablissements_df['Soleil'] = etablissements_df['ETBL_DESC_FR'].str.contains(r'\bsoleil(?:s)?\b', case=False).astype(int)
etablissements_df['Foret'] = etablissements_df['ETBL_DESC_FR'].str.contains(r'\bforet(?:s)?\b', case=False).astype(int)
etablissements_df['Mer'] = etablissements_df['ETBL_DESC_FR'].str.contains(r'\bmer(?:s)?\b', case=False).astype(int)
etablissements_df['Hebergement'] = etablissements_df['ETBL_DESC_FR'].str.contains(r'\bhebergement(?:s)?\b', case=False).astype(int)

# 3. Création de genre_zone 

In [32]:
territoires_df

,ETBL_ID,TERR_GENRE,TERR_ZONE_ID,TERR_ZONE_NOM_FR,TERR_ZONE_NOM_EN,TERR_ZONE_GENRE_ID,TERR_ZONE_GENRE_FR,TERR_ZONE_GENRE_EN
0,838721,principal,292120028,Le mont Royal,The Mont Royal,432971,Quartier / secteur,District / Area
1,838721,principal,21950,Montréal,Montréal,91,Ville,City
2,838721,principal,3432,Montréal,Montréal,81,Région,Area
3,838721,principal,3406,Québec,Quebec,75,Province,Province
4,838774,principal,22830,Boileau,Boileau,91,Ville,City
...,...,...,...,...,...,...,...,...
37173,500016737,principal,3430,Laurentides,Laurentides,81,Région,Area
37174,500016737,principal,3406,Québec,Quebec,75,Province,Province
37175,500016738,principal,21384,Saint-Irénée,Saint-Irénée,91,Ville,City
37176,500016738,principal,3414,Charlevoix,Charlevoix,81,Région,Area


In [34]:
def genre_zone(genre):
    # Filtrage des données selon le genre de zone donné en argument
    filtered_df = territoires_df[territoires_df['TERR_ZONE_GENRE_FR'] == genre]
    
    # Calcul et affichage du nombre d'ETBL_ID par zone de territoire
    result = filtered_df.groupby('TERR_ZONE_NOM_FR')['ETBL_ID'].count()
    print(result)


In [36]:
genre_zone('Ville')

TERR_ZONE_NOM_FR
Acton Vale    4
Adstock       7
Aguanish      5
Akulivik      1
Albanel       3
             ..
Westbury      1
Wickham       1
Windsor       3
Wotton        7
Yamachiche    5
Name: ETBL_ID, Length: 918, dtype: int64


In [38]:
genre_zone('Région')

TERR_ZONE_NOM_FR
Abitibi-Témiscamingue       244
Baie-James                   80
Bas-Saint-Laurent           498
Cantons-de-l'Est           1140
Centre-du-Québec            165
Charlevoix                  762
Chaudière-Appalaches        542
Duplessis                   162
Eeyou Istchee                 8
Gaspésie                    748
Lanaudière                  595
Laurentides                1781
Laval                        22
Manicouagan                 213
Mauricie                    487
Montréal                    541
Montérégie                  303
Nunavik                      64
Outaouais                   447
Québec                     1008
Saguenay-Lac-Saint-Jean     581
Îles-de-la-Madeleine        399
Name: ETBL_ID, dtype: int64


# 4.Création du DataFrame

### D'abord je recupere les regions et leurs ETBL_ID dans territoires_filter

In [321]:
mask_region = territoires_df['TERR_ZONE_GENRE_FR'] == 'Région'
result = territoires_df[mask_region]
territoires_filter =result[['TERR_ZONE_NOM_FR', 'ETBL_ID']]
territoires_filter

,TERR_ZONE_NOM_FR,ETBL_ID
2,Montréal,838721
5,Outaouais,838774
9,Montréal,838885
13,Montréal,838963
17,Montréal,839010
...,...,...
37163,Québec,500016734
37166,Saguenay-Lac-Saint-Jean,500016735
37170,Montréal,500016736
37173,Laurentides,500016737


### Ensuite, je recupere les classifications avec CARACT_ATTRB_NOM_FR et ETBL_ID

In [324]:
caracteristiques_mask = caracteristiques_df['CARACT_NOM_FR'] == 'Classification'
result = caracteristiques_df[caracteristiques_mask]
caracteristiques_filtrer = result[['CARACT_ATTRB_NOM_FR', 'ETBL_ID']]
caracteristiques_filtrer

,CARACT_ATTRB_NOM_FR,ETBL_ID
26,2 étoiles,838721
58,2 étoiles,838774
90,3 étoiles,838885
114,3 étoiles,838963
148,1 étoile supérieur,839010
...,...,...
341227,En cours d'évaluation,500016734
341235,En cours d'évaluation,500016735
341259,En cours d'évaluation,500016736
341281,En cours d'évaluation,500016737


### Fusion des DataFrames

In [430]:
merged_df = pd.merge(territoires_filter, caracteristiques_filtrer, on='ETBL_ID', how='left')
# Compter le nombre d'ETBL_ID par région et par type de classification
merged_df = merged_df.groupby(['TERR_ZONE_NOM_FR', 'CARACT_ATTRB_NOM_FR']).agg(Nombre_ETBL_ID=('ETBL_ID', 'count')).reset_index()
# Remplacer les NaN par 0
merged_df['Nombre_ETBL_ID'] = merged_df['Nombre_ETBL_ID'].fillna(0)
merged_df

,TERR_ZONE_NOM_FR,CARACT_ATTRB_NOM_FR,Nombre_ETBL_ID
0,Abitibi-Témiscamingue,0 étoile,23
1,Abitibi-Témiscamingue,1 étoile,36
2,Abitibi-Témiscamingue,2 soleils,1
3,Abitibi-Témiscamingue,2 étoiles,84
4,Abitibi-Témiscamingue,3 soleils,5
...,...,...,...
222,Îles-de-la-Madeleine,3 soleils,6
223,Îles-de-la-Madeleine,3 étoiles,205
224,Îles-de-la-Madeleine,4 soleils,5
225,Îles-de-la-Madeleine,4 étoiles,25


In [436]:
multi_index = pd.MultiIndex.from_frame(merged_df[['TERR_ZONE_NOM_FR','CARACT_ATTRB_NOM_FR','Nombre_ETBL_ID']])
merged_df.index = multi_index
merged_df.drop(columns=['TERR_ZONE_NOM_FR','CARACT_ATTRB_NOM_FR','Nombre_ETBL_ID'], inplace=True)
merged_df

Empty DataFrame
Columns: []
Index: [(Abitibi-Témiscamingue, 0 étoile, 23), (Abitibi-Témiscamingue, 1 étoile, 36), (Abitibi-Témiscamingue, 2 soleils, 1), (Abitibi-Témiscamingue, 2 étoiles, 84), (Abitibi-Témiscamingue, 3 soleils, 5), (Abitibi-Témiscamingue, 3 étoiles, 98), (Abitibi-Témiscamingue, 4 soleils, 2), (Abitibi-Témiscamingue, 4 étoiles, 52), (Abitibi-Témiscamingue, 5 étoiles, 6), (Abitibi-Témiscamingue, En cours d'évaluation, 12), (Baie-James, 0 étoile, 2), (Baie-James, 1 soleil, 1), (Baie-James, 1 étoile, 19), (Baie-James, 2 soleils, 1), (Baie-James, 2 étoiles, 23), (Baie-James, 3 soleils, 4), (Baie-James, 3 étoiles, 9), (Baie-James, 4 soleils, 1), (Baie-James, Non classifié, 19), (Bas-Saint-Laurent, 0 étoile, 14), (Bas-Saint-Laurent, 1 étoile, 53), (Bas-Saint-Laurent, 1 étoile supérieur, 1), (Bas-Saint-Laurent, 2 soleils, 5), (Bas-Saint-Laurent, 2 étoiles, 148), (Bas-Saint-Laurent, 2 étoiles supérieur, 2), (Bas-Saint-Laurent, 3 soleils, 28), (Bas-Saint-Laurent, 3 étoiles, 149), (Bas-Saint-Laurent, 4 soleils, 16), (Bas-Saint-Laurent, 4 étoiles, 29), (Bas-Saint-Laurent, 5 étoiles, 1), (Bas-Saint-Laurent, En cours d'évaluation, 57), (Cantons-de-l'Est, 0 étoile, 8), (Cantons-de-l'Est, 1 soleil, 1), (Cantons-de-l'Est, 1 étoile, 39), (Cantons-de-l'Est, 2 soleils, 7), (Cantons-de-l'Est, 2 étoiles, 162), (Cantons-de-l'Est, 2 étoiles supérieur, 2), (Cantons-de-l'Est, 3 soleils, 59), (Cantons-de-l'Est, 3 étoiles, 439), (Cantons-de-l'Est, 3 étoiles supérieur, 1), (Cantons-de-l'Est, 4 soleils, 83), (Cantons-de-l'Est, 4 étoiles, 115), (Cantons-de-l'Est, 5 soleils, 3), (Cantons-de-l'Est, 5 étoiles, 5), (Cantons-de-l'Est, En cours d'évaluation, 211), (Centre-du-Québec, 0 étoile, 1), (Centre-du-Québec, 1 étoile, 4), (Centre-du-Québec, 2 soleils, 2), (Centre-du-Québec, 2 étoiles, 31), (Centre-du-Québec, 3 soleils, 6), (Centre-du-Québec, 3 étoiles, 56), (Centre-du-Québec, 4 soleils, 9), (Centre-du-Québec, 4 étoiles, 29), (Centre-du-Québec, En cours d'évaluation, 26), (Centre-du-Québec, Non classifié, 1), (Charlevoix, 0 étoile, 3), (Charlevoix, 1 étoile, 25), (Charlevoix, 2 soleils, 9), (Charlevoix, 2 étoiles, 104), (Charlevoix, 2 étoiles supérieur, 2), (Charlevoix, 3 soleils, 29), (Charlevoix, 3 étoiles, 313), (Charlevoix, 4 soleils, 26), (Charlevoix, 4 étoiles, 162), (Charlevoix, 5 étoiles, 6), (Charlevoix, En cours d'évaluation, 100), (Chaudière-Appalaches, 0 étoile, 6), (Chaudière-Appalaches, 1 étoile, 42), (Chaudière-Appalaches, 2 soleils, 1), (Chaudière-Appalaches, 2 étoiles, 130), (Chaudière-Appalaches, 3 soleils, 33), (Chaudière-Appalaches, 3 étoiles, 199), (Chaudière-Appalaches, 4 soleils, 23), (Chaudière-Appalaches, 4 étoiles, 47), (Chaudière-Appalaches, 5 soleils, 1), (Chaudière-Appalaches, 5 étoiles, 4), (Chaudière-Appalaches, En cours d'évaluation, 64), (Duplessis, 0 étoile, 4), (Duplessis, 1 étoile, 24), (Duplessis, 2 soleils, 6), (Duplessis, 2 étoiles, 51), (Duplessis, 2 étoiles supérieur, 2), (Duplessis, 3 soleils, 12), (Duplessis, 3 étoiles, 51), (Duplessis, 4 soleils, 2), (Duplessis, 4 étoiles, 18), (Duplessis, 5 étoiles, 2), (Duplessis, En cours d'évaluation, 5), (Duplessis, Non classifié, 8), (Eeyou Istchee, 1 étoile, 1), (Eeyou Istchee, 2 étoiles, 2), (Eeyou Istchee, 3 étoiles, 2), (Eeyou Istchee, 3 étoiles supérieur, 1), (Eeyou Istchee, En cours d'évaluation, 2), (Gaspésie, 0 étoile, 20), (Gaspésie, 1 étoile, 82), (Gaspésie, 2 soleils, 6), (Gaspésie, 2 étoiles, 236), (Gaspésie, 2 étoiles supérieur, 1), (Gaspésie, 3 soleils, 50), ...]

[227 rows x 0 columns]

### Utilisation de l'indexation hiérarchique pour afficher le nombre d'ETBL_ID pour chaque type de classification dans chacune des régions

In [366]:
multi_index = pd.MultiIndex.from_frame(merged_df[['TERR_ZONE_NOM_FR','CARACT_ATTRB_NOM_FR','Nombre_ETBL_ID']])
merged_df.index = multi_index
merged_df.drop(columns=['TERR_ZONE_NOM_FR','CARACT_ATTRB_NOM_FR','Nombre_ETBL_ID'], inplace=True)
merged_df

Empty DataFrame
Columns: []
Index: [(Abitibi-Témiscamingue, 0 étoile, 23), (Abitibi-Témiscamingue, 1 étoile, 36), (Abitibi-Témiscamingue, 2 soleils, 1), (Abitibi-Témiscamingue, 2 étoiles, 84), (Abitibi-Témiscamingue, 3 soleils, 5), (Abitibi-Témiscamingue, 3 étoiles, 98), (Abitibi-Témiscamingue, 4 soleils, 2), (Abitibi-Témiscamingue, 4 étoiles, 52), (Abitibi-Témiscamingue, 5 étoiles, 6), (Abitibi-Témiscamingue, En cours d'évaluation, 12), (Baie-James, 0 étoile, 2), (Baie-James, 1 soleil, 1), (Baie-James, 1 étoile, 19), (Baie-James, 2 soleils, 1), (Baie-James, 2 étoiles, 23), (Baie-James, 3 soleils, 4), (Baie-James, 3 étoiles, 9), (Baie-James, 4 soleils, 1), (Baie-James, Non classifié, 19), (Bas-Saint-Laurent, 0 étoile, 14), (Bas-Saint-Laurent, 1 étoile, 53), (Bas-Saint-Laurent, 1 étoile supérieur, 1), (Bas-Saint-Laurent, 2 soleils, 5), (Bas-Saint-Laurent, 2 étoiles, 148), (Bas-Saint-Laurent, 2 étoiles supérieur, 2), (Bas-Saint-Laurent, 3 soleils, 28), (Bas-Saint-Laurent, 3 étoiles, 149), (Bas-Saint-Laurent, 4 soleils, 16), (Bas-Saint-Laurent, 4 étoiles, 29), (Bas-Saint-Laurent, 5 étoiles, 1), (Bas-Saint-Laurent, En cours d'évaluation, 57), (Cantons-de-l'Est, 0 étoile, 8), (Cantons-de-l'Est, 1 soleil, 1), (Cantons-de-l'Est, 1 étoile, 39), (Cantons-de-l'Est, 2 soleils, 7), (Cantons-de-l'Est, 2 étoiles, 162), (Cantons-de-l'Est, 2 étoiles supérieur, 2), (Cantons-de-l'Est, 3 soleils, 59), (Cantons-de-l'Est, 3 étoiles, 439), (Cantons-de-l'Est, 3 étoiles supérieur, 1), (Cantons-de-l'Est, 4 soleils, 83), (Cantons-de-l'Est, 4 étoiles, 115), (Cantons-de-l'Est, 5 soleils, 3), (Cantons-de-l'Est, 5 étoiles, 5), (Cantons-de-l'Est, En cours d'évaluation, 211), (Centre-du-Québec, 0 étoile, 1), (Centre-du-Québec, 1 étoile, 4), (Centre-du-Québec, 2 soleils, 2), (Centre-du-Québec, 2 étoiles, 31), (Centre-du-Québec, 3 soleils, 6), (Centre-du-Québec, 3 étoiles, 56), (Centre-du-Québec, 4 soleils, 9), (Centre-du-Québec, 4 étoiles, 29), (Centre-du-Québec, En cours d'évaluation, 26), (Centre-du-Québec, Non classifié, 1), (Charlevoix, 0 étoile, 3), (Charlevoix, 1 étoile, 25), (Charlevoix, 2 soleils, 9), (Charlevoix, 2 étoiles, 104), (Charlevoix, 2 étoiles supérieur, 2), (Charlevoix, 3 soleils, 29), (Charlevoix, 3 étoiles, 313), (Charlevoix, 4 soleils, 26), (Charlevoix, 4 étoiles, 162), (Charlevoix, 5 étoiles, 6), (Charlevoix, En cours d'évaluation, 100), (Chaudière-Appalaches, 0 étoile, 6), (Chaudière-Appalaches, 1 étoile, 42), (Chaudière-Appalaches, 2 soleils, 1), (Chaudière-Appalaches, 2 étoiles, 130), (Chaudière-Appalaches, 3 soleils, 33), (Chaudière-Appalaches, 3 étoiles, 199), (Chaudière-Appalaches, 4 soleils, 23), (Chaudière-Appalaches, 4 étoiles, 47), (Chaudière-Appalaches, 5 soleils, 1), (Chaudière-Appalaches, 5 étoiles, 4), (Chaudière-Appalaches, En cours d'évaluation, 64), (Duplessis, 0 étoile, 4), (Duplessis, 1 étoile, 24), (Duplessis, 2 soleils, 6), (Duplessis, 2 étoiles, 51), (Duplessis, 2 étoiles supérieur, 2), (Duplessis, 3 soleils, 12), (Duplessis, 3 étoiles, 51), (Duplessis, 4 soleils, 2), (Duplessis, 4 étoiles, 18), (Duplessis, 5 étoiles, 2), (Duplessis, En cours d'évaluation, 5), (Duplessis, Non classifié, 8), (Eeyou Istchee, 1 étoile, 1), (Eeyou Istchee, 2 étoiles, 2), (Eeyou Istchee, 3 étoiles, 2), (Eeyou Istchee, 3 étoiles supérieur, 1), (Eeyou Istchee, En cours d'évaluation, 2), (Gaspésie, 0 étoile, 20), (Gaspésie, 1 étoile, 82), (Gaspésie, 2 soleils, 6), (Gaspésie, 2 étoiles, 236), (Gaspésie, 2 étoiles supérieur, 1), (Gaspésie, 3 soleils, 50), ...]

[227 rows x 0 columns]

## Sélectionnons le nombre de ETBL_ID pour la region Abitibi-Témiscamingue et qui ont 0 étoile

In [443]:
merged_df.loc[pd.IndexSlice['Abitibi-Témiscamingue','0 étoile' ]]

""
Nombre_ETBL_ID
23


## 5. Affichage du nom d'établissement (ETBL_NOM_FR) avec le plus d'activités disponibles


In [407]:
etablissements_df

,ETBL_ID,ETBL_NOM_FR,ETBL_NOM_EN,ETBL_DESC_FR,ETBL_DESC_EN,ETBL_RESERVABLE
0,838721,LE ROCKLEDGE,LE ROCKLEDGE,"Imaginez un séjour dans l'intimité, le luxe et...","Imagine a stay in the intimacy, luxury and inc...",0
1,838774,AUBERGE DU LAC-COMMANDANT,AUBERGE DU LAC-COMMANDANT,Ce chalet rustique pouvant accueillir jusqu¿à ...,"This rustic, 12-bedroom cottage on the shores ...",0
2,838885,UNIVERSITÉ CONCORDIA,UNIVERSITÉ CONCORDIA,La résidence des S¿urs grises est située au c¿...,Welcome to Concordia University's Grey Nuns Re...,0
3,838963,LE GÎTE DU PARC LAFONTAINE,LE GÎTE DU PARC LAFONTAINE,"Auberge de jeunesse située sur le Plateau, en ...","Hostel located in the heart of the Plateau, ne...",0
4,839010,PAVILLON JEAN-XXIII,PAVILLON JEAN-XXIII,Cet hôtel de 17 chambres privées situé à deux ...,"Charming hotel with 17 private rooms, at the f...",0
...,...,...,...,...,...,...
10741,500016734,LE COSY DU LAC,LE COSY DU LAC,,,0
10742,500016735,COOLBOX LE VALINOUËT,COOLBOX LE VALINOUËT,,,0
10743,500016736,CHEZ NOUS CHEZ VOUS,CHEZ NOUS CHEZ VOUS,,,0
10744,500016737,LE LAC POPE,LE LAC POPE,,,0


In [427]:
# Récupérer les valeurs uniques
unique_values = caracteristiques_df['CARACT_ATTRB_NOM_FR'].unique()
unique_values


array(['Golf à moins de 10 km', 'Ski de fond à moins de 10 km',
       "Théâtre d'été à moins de 10 km", 'Vélo',
       'Accès Internet : sans fil gratuit', 'Ascenseur',
       'Établissement entièrement non-fumeurs', 'Laveuse / sécheuse',
       'Literie incluse', 'Remise pour bicyclettes', 'Remise pour skis',
       'Stationnement extérieur gratuit',
       'Stationnement intérieur gratuit', 'Réservations nécessaires',
       'Accès Internet : haute vitesse', 'Accès Internet : sans fil',
       'Cuisinette', 'Micro-ondes', 'Salle de bain privée', 'Sans Tapis',
       'Téléphone', 'Téléviseur', 'Unité pour non-fumeur',
       'appartement~, chalet~ ou maison~', 'Visa', 'MasterCard/Euro Card',
       '2 étoiles', "Capacité maximale de l'unité la plus petite",
       "Capacité maximale de l'unité la plus grande",
       "Maximum pour l'unité la moins chère",
       "Maximum pour l'unité la plus chère", 'Motoneige',
       'Patinage sur glace', 'Plage', 'Quad / VTT', 'Randonnée pédestre'